In [5]:
# Cellule 1 : Importations et Configuration (Sécurisée)

# Imports nécessaires
import sqlite3
import pandas as pd
import os
from sqlalchemy import create_engine, text
from snowflake.sqlalchemy import URL
from snowflake.connector.errors import OperationalError

# NOUVEL IMPORT : Permet de charger les variables depuis le fichier .env
from dotenv import load_dotenv

# ======================================================================
#                            CONFIGURATION
# ======================================================================

# 1. Charger les variables du fichier .env pour les rendre disponibles
load_dotenv() 

# 2. Récupérer les valeurs SÉCURISÉES via os.getenv()
# A. Infos sur la source
DB_PATH = os.getenv('DB_PATH')

# Nom que la table aura dans Snowflake
SNOWFLAKE_TARGET_TABLE = 'Orders'

# B. Infos de connexion à Snowflake (Les secrets ne sont plus en clair dans le code)
SNOWFLAKE_USER = os.getenv('SNOWFLAKE_USER')
SNOWFLAKE_PASSWORD = os.getenv('SNOWFLAKE_PASSWORD')
SNOWFLAKE_ACCOUNT = os.getenv('SNOWFLAKE_ACCOUNT') 
SNOWFLAKE_WAREHOUSE = os.getenv('SNOWFLAKE_WAREHOUSE')
SNOWFLAKE_DATABASE = os.getenv('SNOWFLAKE_DATABASE')
SNOWFLAKE_SCHEMA = os.getenv('SNOWFLAKE_SCHEMA')      

print("Configuration chargée de manière sécurisée et prête à être utilisée.")

Configuration chargée de manière sécurisée et prête à être utilisée.


In [6]:
# --- Préparation de la Connexion SQLAlchemy ---

# On construit l'adresse complète pour se connecter à Snowflake
snowflake_url = URL(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE, 
    schema=SNOWFLAKE_SCHEMA
)

try:
    # On crée le moteur qui gère la communication
    engine = create_engine(snowflake_url)
    
    # On ouvre une connexion SÛRE qui se ferme automatiquement grâce au 'with'
    with engine.connect() as conn_snowflake:
        print(f"✅ Connexion à Snowflake établie sur {SNOWFLAKE_DATABASE}.{SNOWFLAKE_SCHEMA}.")

        # --- Étape 1 : Création de la Base de Données et du Schéma ---
        print("\n--- Préparation de la DB et du Schéma dans Snowflake ---")

        # Crée la DB et le Schéma s'ils n'existent pas
        conn_snowflake.execute(text(f"CREATE DATABASE IF NOT EXISTS {SNOWFLAKE_DATABASE}"))
        conn_snowflake.execute(text(f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"))
        
        # S'assure d'utiliser le bon entrepôt
        conn_snowflake.execute(text(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}"))
        conn_snowflake.execute(text("COMMIT")) # Enregistre les changements DDL
        
        print(f"   -> Contexte DDL prêt.")

except OperationalError as e:
    print(f"🛑 Erreur critique : Problème d'identifiants ou de réseau. {e}")
except Exception as e:
    print(f"🛑 Erreur lors de la préparation : {e}")

# L'engine et la connexion sont maintenant fermés, ce qui est normal ici.

✅ Connexion à Snowflake établie sur ETL_PROJECT_DB.SALES_SCHEMA.

--- Préparation de la DB et du Schéma dans Snowflake ---
   -> Contexte DDL prêt.


In [7]:
# --- Étape 2 : Extraction et Transformation (E & T) ---

order_df = None # Initialisation du DataFrame

if not os.path.exists(DB_PATH):
    print(f"\n🛑 Erreur: Fichier de base de données SQLite non trouvé à l'adresse: {DB_PATH}")
    order_df = pd.DataFrame() 
else:
    try:
        print("\n--- Extraction de SQLite vers DataFrame ---")
        conn_sqlite = sqlite3.connect(DB_PATH)
        
        # Requête pour extraire toutes les données de la table order
        query = "SELECT * FROM orders;"
        order_df = pd.read_sql_query(query, conn_sqlite)
        
        print(f"   -> DataFrame 'Invoice' chargé avec {len(order_df)} lignes.")

    except sqlite3.Error as e:
        print(f"🛑 Erreur SQLite lors de l'extraction : {e}")
        order_df = pd.DataFrame()
    finally:
        if 'conn_sqlite' in locals() and conn_sqlite:
            conn_sqlite.close()

if not order_df.empty:
    display(order_df.head())
else:
    print("Aucune donnée à charger.")


--- Extraction de SQLite vers DataFrame ---
   -> DataFrame 'Invoice' chargé avec 100 lignes.


,id,product_id,customer_id,quantity,unit_price,sold_at
0,1,1057,60,3,9.67,2025-08-29 18:57:36+00:00
1,2,1075,61,1,7.97,2025-02-01 14:40:07+00:00
2,3,1023,19,1,228.24,2024-08-23 15:48:34+00:00
3,4,1088,51,2,39.84,2025-04-07 11:44:27+00:00
4,5,1083,21,2,37.70,2025-05-19 22:12:04+00:00


In [8]:
# --- Étape 3 : Chargement (L) dans Snowflake ---

if order_df is not None and not order_df.empty: 
    
    # Recréer l'adresse de connexion car l'ancienne a été fermée
    snowflake_url = URL(
        user=SNOWFLAKE_USER,
        password=SNOWFLAKE_PASSWORD,
        account=SNOWFLAKE_ACCOUNT,
        warehouse=SNOWFLAKE_WAREHOUSE,
        database=SNOWFLAKE_DATABASE, 
        schema=SNOWFLAKE_SCHEMA
    )

    try:
        # On recrée le moteur, puis on ouvre une nouvelle connexion SÛRE avec 'with'
        engine = create_engine(snowflake_url)
        with engine.connect() as conn_snowflake:
            
            print("\n--- Début du Chargement du DataFrame vers Snowflake ---")
            
            # On crée le nom complet de la table: "DB"."SCHEMA".TABLE_NAME (IMPORTANT pour Snowflake)
            qualified_table_name = SNOWFLAKE_TARGET_TABLE
            
            # Chargement des données !
            order_df.to_sql(
                qualified_table_name, # On utilise le nom COMPLET pour éviter les erreurs de contexte
                con=conn_snowflake, 
                if_exists='replace', # Crée la table, ou la remplace si elle existe
                index=False,         # N'insère pas l'index du DataFrame dans la table
                chunksize=10000,
                method='multi',
            )
            
            conn_snowflake.execute(text("COMMIT")) # Confirme l'insertion des données
            
            print(f"🎉 Chargement de la table '{SNOWFLAKE_TARGET_TABLE}' terminé avec succès !")

    except Exception as e:
        print(f"\n🛑 Erreur critique lors du chargement des données dans Snowflake : {e}")
        
else:
    print("🛑 Chargement annulé car le DataFrame des commandes est vide.")

print("Fermeture de la connexion Snowflake effectuée (grâce à la clause 'with').")


--- Début du Chargement du DataFrame vers Snowflake ---
🎉 Chargement de la table 'Orders' terminé avec succès !
Fermeture de la connexion Snowflake effectuée (grâce à la clause 'with').
